<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/II.%20Introduccion_Ciencia_Datos/EjercicioAdvertising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Regresión  - Ejercicio Advertising

El objetivo es usar regresión probablemente multivariada para entender como los gastos en anuncios impactan las ventas (variable a predecir es **sales**).

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [70]:
url = "https://raw.githubusercontent.com/eduardool/MaestriaCienciaDatos/main/datasets/advertising_regresion.csv"

In [71]:
datos = pd.read_csv(url)

In [72]:
datos.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [73]:
datos=datos.drop("Unnamed: 0",axis=1)

In [74]:
datos.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [75]:
datos.shape

(200, 4)

In [76]:
datos.info()#no se tienen datos nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [77]:
datos.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,200.0,147.0425,85.854236,0.7,74.375,149.75,218.825,296.4
radio,200.0,23.2640,14.846809,0.0,9.975,22.90,36.525,49.6
newspaper,200.0,30.5540,21.778621,0.3,12.750,25.75,45.100,114.0
sales,200.0,14.0225,5.217457,1.6,10.375,12.90,17.400,27.0


In [78]:
datos.columns

Index(['TV', 'radio', 'newspaper', 'sales'], dtype='object')

**Preparamos los datos para los algoritmos de ML**

In [79]:
X=datos[['TV','radio','newspaper']]
y=datos['sales']

Separación de los datos en conjunto de entrenamiento y prueba

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=100)

In [82]:
X_entrenamiento.shape

(160, 3)

In [83]:
X_prueba.shape

(40, 3)

Estandarización del conjunto de entrenamiento

In [84]:
from sklearn.preprocessing import StandardScaler

In [85]:
X_preparado=StandardScaler().fit_transform(X_entrenamiento)

In [86]:
#X_preparado

#MODELO DE REGRESION LINEAL

In [87]:
from sklearn.linear_model import LinearRegression

In [88]:
reg_lin = LinearRegression()

reg_lin.fit(X_preparado, y_entrenamiento)

LinearRegression()

In [89]:
# prueba con algunas instancias pequeñas
pocos_X = X_entrenamiento.iloc[:5]
pocos_y = y_entrenamiento.iloc[:5]

pocos_datos_preparados = StandardScaler().fit_transform(pocos_X)

print("Predicciones:", reg_lin.predict(pocos_datos_preparados))

Predicciones: [20.90463152 12.69013752 13.89650424 13.31565588  8.18682084]


In [90]:
print("y real:", list(pocos_y))

y real: [17.1, 14.1, 14.5, 12.2, 13.2]


In [93]:
predicciones_X = reg_lin.predict(X_preparado)

In [97]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_entrenamiento, predicciones_X)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.7518885936753472

In [95]:
from sklearn import metrics

In [100]:
metrics.mean_squared_error(y_entrenamiento, predicciones_X)

3.0691136446497858

In [103]:
print('MAE:', metrics.mean_absolute_error(y_entrenamiento, predicciones_X))
print('MSE:', metrics.mean_squared_error(y_entrenamiento, predicciones_X))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_entrenamiento, predicciones_X)))

MAE: 1.3491729189156043
MSE: 3.0691136446497858
RMSE: 1.7518885936753472


Separar $X$ y $y$

Separar conjunto de entrenamiento (80\%) y prueba (20\%), random_state = 100

Calcular y realizar la escala de variables sobre el conjunto entrenamiento.

Usar validación cruzada (5-folds) para comparar los algoritmos de regresión lineal multivariada y Random forest. En ambos algoritmos usar un random_state = 100. Cual algoritmo es mejor?

Realizar el afinamiento del algoritmo que tuvo mejor desempeño (usar GridSearchCV).


Reportar el RMSE del algoritmo afinado sobre el conjunto de prueba (no olvidarse de antes aplicar la escala calculada sobre el conjunto de entrenamiento).